In [1]:
#!pip install Augmentor 

In [2]:
#ライブラリをインポート
#標準ライブラリ
import os
import pathlib

#サードパーティー製
import Augmentor
import pandas as pd

In [ ]:
list(Path().iterdir())

# 概要

Augmenterを使用して、オフライン拡張をおこなう．<br>

In [1]:
"""
変更箇所
"""
#訓練画像のパス
train_path = "Train"
#水増し処理後の画像の保存先のパス
augmentation_output_path = "Train/output"
#名前変更後の画像の出力先のパス
rename_output_path = "output_after_rename_img_9999"

In [4]:
#出力先の画像の名前を求める形に変更
def rename_path(augmentation_output_path, rename_output_path):
    """
    水増し処理をした画像のパス名を変換させる関数
    
    augmentation_output_path : str
        水増し処理後の画像群のパス
    rename_output_path : str
        リネーム後の画像を出力するパス
    """
    
    #画像のパスを取得
    list_path = list(pathlib.Path(augmentation_output_path).iterdir())
    #画像のパス名をstr型に変更
    list_path = [str(path) for path in list_path]
    
    #DF化
    df_list_path = pd.DataFrame(list_path)
    
    #それぞれのパスのフルーツ名を取得
    list_fruit_name = list(df_list_path[0].str.split("\\", expand = True).iloc[:, -1].str.split("_", expand = True).iloc[:, 2])
    display()
    
    #パスと果物名の列があるdfを作成
    df_path_and_fruitname = pd.DataFrame([list_path, list_fruit_name]).T
    df_path_and_fruitname.columns = ["パス", "果物名"]
    
    #パス名の変換を実施
    apple_count = 0
    tomato_count = 0
    cherry_count = 0
    for path, fruit_name in df_path_and_fruitname.values:
        #この書き方あまり好きではないが、、、時短のため
        #リンゴの場合
        if(fruit_name == "apple"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(apple_count) + ".jpg"))
            #リンゴ＋１
            apple_count += 1
        #トマトの場合
        elif(fruit_name == "tomato"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(tomato_count) + ".jpg"))
            #トマト＋１
            tomato_count += 1
        #サクランボの場合
        elif(fruit_name == "cherry"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(cherry_count) + ".jpg"))
            #さくらんぼ＋１
            cherry_count += 1

In [5]:
# 画像の回転＋明るさ+弾性歪み
def rotate_and_adjust_skrw_erasing(img_path, prob = 0.2, max_left = 360, max_right = 360, min_factor_adjust = 0.3, max_factor_adjust = 0.8, \
                                 mag = 1, grid = 5, mag_d = 3):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    grid:int
        弾性歪み時の長方形の数
    mag_d:int
        弾性歪みの程度
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    p.skew(probability=prob, magnitude=0.1)
    p.skew(probability=prob, magnitude=0.2)
    p.skew(probability=prob, magnitude=0.3)
    #p.skew(probability=prob, magnitude=0.4)
    #p.skew(probability=prob, magnitude=0.5)
    #p.skew(probability=prob, magnitude=0.6)
    #p.skew(probability=prob, magnitude=0.7)
    #p.skew(probability=prob, magnitude=0.8)
    #p.skew(probability=prob, magnitude=0.9)
    #p.skew(probability=prob, magnitude=1)
    #くりぬき実施
    p.crop_random(probability=prob, percentage_area=0.7, randomise_percentage_area=False)
    #反転
    p.flip_random(probability=prob)
    #弾性歪み
    p.random_distortion(probability=prob, grid_width = 5, grid_height = 5, magnitude=5)
    p.random_distortion(probability=prob, grid_width = 6, grid_height = 6, magnitude=6)
    #p.random_distortion(probability=prob, grid_width = 7, grid_height = 7, magnitude=7)
    #p.random_distortion(probability=prob, grid_width = 8, grid_height = 8, magnitude=8)
    #p.random_distortion(probability=prob, grid_width = 9, grid_height = 9, magnitude=9)
    #彩度
    p.random_color(probability=prob, min_factor=0.1, max_factor=1.0)
    #コントラスト
    p.random_contrast(probability=prob, min_factor=0.1, max_factor=1.0)
    #情報損失
    p.random_erasing(probability=prob, rectangle_area=0.4)
    p.random_erasing(probability=prob, rectangle_area=0.2)
    
    p.sample(9999)

In [6]:
rotate_and_adjust_skrw_erasing(train_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=204x125 at 0x20FE3855700>:  40%|▍| 4012/9999 [02:25<03:08, 31.70 Sample

In [7]:
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

In [8]:
i = [1*10]

In [9]:
print(i)